In [385]:
# I.
# -*- coding: utf-8 -*-

import matplotlib.pyplot as plt
import matplotlib as mpl
import pandas as pd

import geopandas as gpd
from area import area
import numpy as np


In [386]:
# Reading the main reference df: wlands.

wlands = pd.read_csv('/home/hmdg/Documents/Data_Science/Repos/Lands/data/wlands.csv', 
                      header=0, encoding='iso8859_1')

wlands.columns
# Reorder the columns 
wlands = wlands[['name_w','region_country','type', 'value','year', 'continent', 
                 'pop_est', 'geometry','number', 'iso_a3', 'gdp_md_est', 'footnotes', 
                 'source', 'general_source']]

Let's inspect the data.

In [391]:
wlands_16 = wlands[wlands.year==2016]
wcount = wlands_16.groupby(['name_w']).count()
wun = wcount['type'].unique()

- This list {{wun}} are the differents values for lands types per country.
- Those over 8, are more than expected; and under 8, are missing.

We'll work at first this with surplus values.

In [392]:
wcount = wlands_16.groupby(['name_w']).count()
wcount[wcount['type']>8]

,region_country,type,value,year,continent,pop_est,geometry,number,iso_a3,gdp_md_est,footnotes,source,general_source
name_w,,,,,,,,,,,,,
China,16,16,16,16,16,16,16,16,16,16,13,16,16
Netherlands,11,11,11,11,11,11,11,11,11,11,7,11,11


Are China and Netherlands this that has extra values.
Let's zoom them.

In [393]:
wlands_16[(wlands_16['name_w'].isin(['China','Netherlands']))
         & (wlands_16['type']=='Land area (thousand hectares)')]

,name_w,region_country,type,value,year,continent,pop_est,geometry,number,iso_a3,gdp_md_est,footnotes,source,general_source
3151,Netherlands,Netherlands,Land area (thousand hectares),3369.0,2016,Europe,17084719,"POLYGON ((6.905139601274129 53.48216217713065,...",528,NLD,870800.0,Estimate.,Food and Agriculture Organization of the Unite...,https://data.un.org/
3170,Netherlands,Netherlands Antilles [former],Land area (thousand hectares),80.0,2016,Europe,17084719,"POLYGON ((6.905139601274129 53.48216217713065,...",530,NLD,870800.0,NaN,Food and Agriculture Organization of the Unite...,https://data.un.org/
3389,China,China,Land area (thousand hectares),938821.0,2016,Asia,1379302771,MULTIPOLYGON (((109.4752095886636 18.197700913...,156,CHN,21140000.0,"Estimate.;For statistical purposes, the data f...",Food and Agriculture Organization of the Unite...,https://data.un.org/
3410,China,"China, Hong Kong SAR",Land area (thousand hectares),105.0,2016,Asia,1379302771,MULTIPOLYGON (((109.4752095886636 18.197700913...,344,CHN,21140000.0,Estimate.,Food and Agriculture Organization of the Unite...,https://data.un.org/
3421,China,"China, Macao SAR",Land area (thousand hectares),3.0,2016,Asia,1379302771,MULTIPOLYGON (((109.4752095886636 18.197700913...,446,CHN,21140000.0,NaN,Food and Agriculture Organization of the Unite...,https://data.un.org/


- The 'name_w' column is the reference, and the 'region_country' is the name comming from the Land df.
- In both cases, we can see that even that the values of every type of land change, the geometry is the same for every country named in 'name_w'.
- Hence, in the map plot, it'll be not possible to draw the different locations.

So, this extra columns are going to be excluded.


In [394]:
wlands_16 = wlands_16[~wlands_16['region_country'].isin(['Netherlands Antilles [former]', 
                                              'China, Hong Kong SAR',
                                             'China, Macao SAR'])]

Let's work just with the year 2016, and do some math.

In [395]:
countries = len(wlands_16['name_w'].unique())
expected = 8 * countries
actual_registers = wlands_16.shape[0]
diffe = expected - actual_registers      

- Notes:
    - There are {{countries}} differentes countries.
    - If every country has 8 differents types of land, we are specting to have {{expected}} registers in the dataframe; and we actually have {{actual_registers}} registers.
    - This means, are {{diffe}} missing.


In [396]:
wc = wlands_16.groupby(['type']).count()
wc.year.sort_values(ascending=False)

type
Forest cover (thousand hectares)                                                       170
Land area (thousand hectares)                                                          168
Forest cover (% of total land area)                                                    168
Arable land (thousand hectares)                                                        167
Arable land (% of total land area)                                                     166
Important sites for terrestrial biodiversity protected (% of total sites protected)    165
Permanent crops (thousand hectares)                                                    164
Permanent crops (% of total land area)                                                 163
Name: year, dtype: int64

In [418]:
# Missing records from which countries?

wo = wlands_16.groupby(['name_w']).count()
wless = 8 - wo.type[wo.type<8]
print(wless, 'total missing: ', wless.sum() )

#wless.unstack().index
#wlist = wless.unstack().index.tolist()
wlist = wless.index.tolist() # List of countries with missing registers
w_16=wlands_16[wlands_16['name_w'].isin(wlist)]


name_w
Djibouti                    2
Falkland Is.                4
Greenland                   4
Guyana                      1
Iceland                     2
Jordan                      1
Peru                        1
S. Sudan                    6
Sudan                       4
United States of America    1
W. Sahara                   3
Name: type, dtype: int64 total missing:  29


In [419]:
# Which has no Land Area?

wo = w_16.groupby(['name_w','type']).count()
wo = wo.reset_index(level=['type'])
wo = wo.pivot(columns='type', values='value')
wo.style.highlight_null(null_color='yellow')


type,Arable land (% of total land area),Arable land (thousand hectares),Forest cover (% of total land area),Forest cover (thousand hectares),Important sites for terrestrial biodiversity protected (% of total sites protected),Land area (thousand hectares),Permanent crops (% of total land area),Permanent crops (thousand hectares)
name_w,,,,,,,,
Djibouti,1,1,1,1,1,1,nan,nan
Falkland Is.,nan,nan,1,1,1,1,nan,nan
Greenland,nan,nan,1,1,1,1,nan,nan
Guyana,1,1,1,1,nan,1,1,1
Iceland,1,1,1,1,1,1,nan,nan
Jordan,1,1,1,1,nan,1,1,1
Peru,1,1,1,1,nan,1,1,1
S. Sudan,nan,nan,nan,1,1,nan,nan,nan
Sudan,nan,1,nan,1,1,nan,nan,1


# !!!!!!!!!ADD THE 29 rows!!!!! and then fille them.

In [354]:
# Let's see every type of missing land:
## 1rst Land area (thousand hectares)

wlands_16[['region_country', 'name_w', 'value', 'type']][(wlands_16.region_country=='South Sudan')]

,region_country,name_w,value,type
4178,South Sudan,S. Sudan,7157.00,Forest cover (thousand hectares)
4179,South Sudan,S. Sudan,33.64,Important sites for terrestrial biodiversity p...


In [121]:
# VII
######## Creción de la tabla de referencia/relación de países

ldown = landun.loc[685:] # Registros de Regiones del mundo
lup = landun.loc[:684] # Registros de Países

# creo DF con los nombres de landun
countries = ldown['region_country'].unique() # Global var
countries_dg = pd.DataFrame(countries, columns=['region_country'])
countries_dg['region_country'] = countries_dg['region_country'].str.lower()
lt = ['region_country', 'bool', 'name_w']
countries_dg = countries_dg.reindex(columns=lt)

# creo DF con los nombres de world
world_df = pd.DataFrame(world['name'].str.strip().str.lower())
lt = ['name', 'bool', 'name_w']
world_df = world_df.reindex(columns=lt)
world_df = world_df.rename(columns={'name':'name_l'})

# Busco los valores coincidentes en landun y world, coincidencias exactas
world_df['bool']=world_df['name_l'].isin(countries_dg['region_country'])
world_df['name_w'] = world_df['name_w'].mask(world_df['bool']==True, world_df['name_l'])
w_nc = world_df[world_df['bool']==False]
w_nc = w_nc.reset_index(drop=True)
w_nc = w_nc.copy()

# Busco los valores NO coincidentes (exactos) en landun y world
countries_dg['bool']=countries_dg['region_country'].isin(world_df['name_l'])
c_nc = countries_dg[countries_dg['bool']==False]
c_nc = c_nc.reset_index(drop=True)
c_nc = c_nc.rename(columns={'region_country':'name_l'})

# Contrasto los valores con coincidencias parciales entre ambas tablas
for nc in range(w_nc.shape[0]):
    pat = w_nc.loc[nc,'name_l']
    c_nc['name_w'] = c_nc['name_w'].mask(c_nc['name_l'].str.contains(pat, regex=False),pat)
       
# Añado las últimas coincidencias a al df original world_df, cuando el name_w NO esta vacio
world_con = pd.concat([world_df[world_df['name_w'].notna()],c_nc[c_nc['name_w'].notna()]], sort=True)

# Confirmo cuales son los países que faltan - No hay concidencia total o parcial
w_faltantes = world_df[~world_df['name_l'].isin(world_con['name_w'])]


# Añado los nombres de los países de doc. Excel (hecho a mano)
pays_faltantes = pd.read_csv('/home/hmdg/Documents/Data_Science/Maps/conversion_world_f.csv', encoding='iso8859_1')#'utf-8-sig')
countries_ref = pd.concat([world_con,pays_faltantes], sort=True)
countries_ref = countries_ref.reset_index(drop=True)
countries_ref = countries_ref.drop(columns=['bool'])
countries_ref

countries_ref.to_csv(r'/home/hmdg/Documents/Data_Science/Lands/countries_ref.csv', index = False)

,name_l,bool,name_w
0,american samoa,False,NaN
1,andorra,False,NaN
2,anguilla,False,NaN
3,antigua and barbuda,False,NaN
4,aruba,False,NaN
5,bahrain,False,NaN
6,barbados,False,NaN
7,bermuda,False,NaN
8,bolivia (plurin. state of),False,NaN
9,"bonaire, st. eustatius & saba",False,NaN


In [9]:
# VII

# Inspección de la dataII - landun con world 

lp = pd.DataFrame(lp) # Viene de la parte I, países faltantes

lp['country'] = lp['country'].str.strip().str.lower()
countries_ref

# Estos son los países que tienen data faltante y que coinciden con los registos de world
pfalt=lp.merge(countries_ref, how='inner', left_on='country', right_on='name_l')

p_invest = pfalt['country']
p_invest = p_invest.copy() 

print('_____________________________________________________________________')

#### Tengo que ver por cada país, cuál es la cantidad de registros que faltan

# cuantos registros debe tener cada pais:
print('Estos son los registros por país (head)')

ldown_=ldown[['type','country', 'year']]
cc = ldown_.groupby(['country']).count()
print(cc.sort_values(by='type', ascending=False).head())

reg_año = ldown['type'].unique().shape[0]
cant_años = ldown['year'].unique().shape[0]

reg_pais = (reg_año*(cant_años-1)) + 1

print('\nPorque? porque cada país tiene 8 registros por 3 años, mas 1 registro del 2018.\n Esto hace un total de: ', str(reg_pais), 'registros por país.\n') # 


ldown['country'] = ldown['country'].str.strip().str.lower()
ldown = ldown.copy()
ldown = ldown.reset_index(drop=True)

# llenaremos la cantidad de registros faltantes con data NaN en esos países
print('\nLlenaremos la cantidad de registros faltantes con data NaN en esos países\n. Cuáles son estos países? (head)')
cf = cc[(cc['type']<25)].sort_values(by='type', ascending=False)

print('_______________________________________________________________________')
print(cf.head())
   
print('\nCuáles de estos estos países coinciden con la tabla de world?\n')

ldownf = ldown['country'].isin(p_invest)
ldownf = ldownf.copy()
ldownf2 = ldown[ldownf]
ldownf2 = ldownf2[['type','country', 'region']]

l2 = ldownf2.groupby(['country','region']).count()
l2 = l2.copy()

print('l2[type]: ', l2['type'])
# Ahora si vamos a llenar las filas que faltan con país, con NaN
print('\nAhora si vamos a llenar las filas que faltan con país, con NaN.')
print(l2['type'].sum())

lpa = [l2.index[p] for p in range(len(l2)) for s in range(25-l2['type'][p])]

lpb = pd.DataFrame(lpa, columns=['country', 'region'])

lpb = lpb.copy()

ldown_ap = pd.concat([ldown,lpb], sort=False)
ldown_ap= ldown_ap.copy()

#print('rows antes de añadir las nuevas: ', ldown.shape, ' y después: ', ldown_ap.shape)

# print('\nejemplo con guyana\n')
# print(ldown_ap[ldown_ap['country']=='guyana'])


_____________________________________________________________________
Estos son los registros por país (head)
             type  year
country                
Afghanistan    25    25
Nigeria        25    25
Mongolia       25    25
Morocco        25    25
Mozambique     25    25

Porque? porque cada país tiene 8 registros por 3 años, mas 1 registro del 2018.
 Esto hace un total de:  25 registros por país.


Llenaremos la cantidad de registros faltantes con data NaN en esos países
. Cuáles son estos países? (head)
_______________________________________________________________________
                          type  year
country                             
United States of America    21    21
Peru                        21    21
Jordan                      21    21
Guyana                      21    21
Andorra                     19    19

Cuáles de estos estos países coinciden con la tabla de world?

l2[type]:  country                      region
djibouti                     262       19

/home/hmdg/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:35: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [10]:
# VIII 

## SOLUCION DE MATRICES

ltype
lyear=ldown['year'].unique()
lyear=lyear[:3] # No tomo en cuenta el año 2018
print(lyear)
p_invest
matriz = []

# Creo el df con todos los valores de type, de los países a los que les falta data

for co in range(p_invest.shape[0]):
    for t in range(ltype.shape[0]):
        for y in range(lyear.shape[0]):
            conj = (ltype[t],lyear[y], p_invest[co])
            matriz.append(conj)
            
matriz = pd.DataFrame(matriz, columns=(['type','year','country']))

# # Creo el df con todos los valores de la región 
region = ldown.groupby(['region','country']).count()
rego = region['value'].reset_index(level=['region','country'])
rego = rego[['region','country']]

# Hago el merge de matriz con region
mat_reg = rego.merge(matriz, on='country',how='inner')

mat_reg = mat_reg[['type', 'year', 'country', 'region']]


# Hago el merge de mat_region con ldown

ldown_m = ldown[['region','country','type', 'year']]
ldown_n = ldown_m.merge(mat_reg, on=['country', 'year','type', 'region'], how='outer', indicator=True)

ldown_n['_merge'].unique() # [left_only, both, right_only]

ld = ldown_n[(ldown_n['_merge']=='right_only')]
ld = ld.reset_index(drop=True)
ld = ld[['region', 'country', 'type', 'year']]

ld

# hago FINALMENTE el concat de ld con la tabla de ldown
ldown_c = pd.concat([ldown, ld], sort=True)

ldown_c = ldown_c.reset_index(drop=True)


[2005 2010 2016]


In [11]:
# IX 

# Llenando los valores faltantes.

# I Land Area faltante 

lland = ldown_c[['country', 'value', 'year', 'type']][(ldown_c['type']=='Land area (thousand hectares)')
        & (ldown_c['value'].isnull())]


# Tenemos el forrest cover% de south sudan y sudan para el año 2016. Añadiremos el Land Area de 

# Sudan
# cada uno y calcularemos el Forrest Cover.
# 1,886,068 km²./ 188606.800 th Ha sudan  --- WIKI https://en.wikipedia.org/wiki/Geography_of_Sudan, 

#ldown_c.columns # ['country', 'footnotes', 'general_source', 'region', 'source', 'type', 'value', 'year']

lsudan = pd.DataFrame({'value':188606.800, 'general_source':'https://en.wikipedia.org/wiki/Geography_of_Sudan'},index=[5581])
ldown_c.update(lsudan)

ldown_c.loc[5581]

# South Sudan

# 619,745 km2 / 61974.500 th Ha South Sudan  --- WIKI https://en.wikipedia.org/wiki/South_Sudan, a partir de 2011....
lssudan = pd.DataFrame({'value':61974.500 , 'general_source':'https://en.wikipedia.org/wiki/Geography_of_Sudan'},index=[5561])
ldown_c.update(lssudan)
ldown_c.loc[5561]

#############

# II Forrest Cover faltante 
lforrest = ldown_c[['country', 'value', 'year', 'type']][(ldown_c['type'].str.contains('Forest cover|Land area')) 
                                                        & (ldown_c['year']==2016)]

lf1 = lforrest['country'][lforrest['value'].isnull()].unique()
lf2 = lforrest[lforrest['country'].isin(lf1)].sort_values(by=['year','country'])

indices = [5578, 5596]
i=0
for n in lf1:
    landsu = lf2['value'][(lf2['country']==n) & (lf2['type']=='Land area (thousand hectares)')]  
    fcovsu = lf2['value'][(lf2['country']==n) & (lf2['type']=='Forest cover (thousand hectares)')] 
    fpers = 100*np.divide(fcovsu,landsu)
    lfsu = pd.DataFrame({'value':fpers.item(), 'footnotes':'Calculated Value'},index=[indices[i]])
    ldown_c.update(lfsu)
    i += 1

ldown_c[['country', 'footnotes', 'type', 'value']][(ldown_c['country'].str.contains('sudan')) & (ldown_c['year']==2016)]


,country,footnotes,type,value
4595,south sudan,Estimate.,Forest cover (thousand hectares),7157.000000
4596,south sudan,NaN,Important sites for terrestrial biodiversity p...,33.640000
4675,sudan,Estimate.,Arable land (thousand hectares),19823.000000
4676,sudan,Estimate.,Permanent crops (thousand hectares),168.000000
4677,sudan,Estimate.,Forest cover (thousand hectares),19036.000000
4678,sudan,NaN,Important sites for terrestrial biodiversity p...,25.000000
5561,south sudan,NaN,Land area (thousand hectares),61974.500000
5564,south sudan,NaN,Arable land (thousand hectares),NaN
5567,south sudan,NaN,Permanent crops (thousand hectares),NaN
5572,south sudan,NaN,Arable land (% of total land area),NaN


In [12]:
# X

# III Arable Lands Faltantes

c_falt = ldown_c['country'][(ldown_c['value'].isnull()) & (ldown_c['year']==2016)].unique()

larable = ldown_c[(ldown_c['type'].str.contains('Land area|Arable land')) & (ldown_c['year']==2016) & (ldown_c['country'].isin(c_falt))]

# Sudan is the onlyone that has the 'Arable land (tho. Ha)'... I'll fill this one first
# Sudan
adsu = larable['value'][(larable['country']=='sudan') & (larable['type']=='Arable land (thousand hectares)')]  
cvsu = larable['value'][(larable['country']=='sudan') & (larable['type']=='Land area (thousand hectares)')] 
ldown_c.update(pd.DataFrame({'value':100*np.divide(adsu, cvsu).item(), 'footnotes':'Calculated Value'},index=[5590]))

# Faltan los valores de Fakland Island, greenland and South Sudan. No tienen ninguno de los valores de Arable.

# Fakland Island
# https://www.cia.gov/library/publications/the-world-factbook/geos/print_fk.html
# agricultural land: 92.4% (2011 est.)
# arable land: 0% (2011 est.) / permanent crops: 0% (2011 est.) / permanent pasture: 92.4% (2011 est.)
# forest: 0% (2011 est.)
# other: 7.6% (2011 est.)

ldown_c.update(pd.DataFrame({'value':0, 'general_source':'https://www.cia.gov/library/publications/the-world-factbook/geos/print_fk.html'},index=[5516]))
ldown_c.update(pd.DataFrame({'value':0, 'general_source':'https://www.cia.gov/library/publications/the-world-factbook/geos/print_fk.html'},index=[5522]))

# Greenland
# https://tradingeconomics.com/greenland/arable-land-percent-of-land-area-wb-data.html
# Agricultural land refers to the share of land area that is arable, under permanent crops, and under permanent pastures.
# Arable land (hectares) 900 ha, 0.9 th Ha
# Arable land (hectares per person) 0.016
# Arable land (% of land area) 0.00219 %

ldown_c.update(pd.DataFrame({'value':0.9/2, 'general_source':'https://tradingeconomics.com/greenland/arable-land-percent-of-land-area-wb-data.html', 'footnotes': 'Agricultural land refers to the share of land area that is arable, under permanent crops, and under permanent pastures.'},index=[5534]))
ldown_c.update(pd.DataFrame({'value':0.00219/2, 'general_source':'https://tradingeconomics.com/greenland/arable-land-percent-of-land-area-wb-data.html'},index=[5540]))

# South Sudan
# https://tradingeconomics.com/south-sudan/agricultural-land-sq-km-wb-data.html
# Agricultural land refers to the share of land area that is arable, under permanent crops, 
# and under permanent pastures.
# Agricultural land (sq. km) 285332 sq. Km - 28533200Ha, 28533.2 th Ha

ldown_c.update(pd.DataFrame({'value':28533.2/2, 'general_source':'https://tradingeconomics.com/greenland/arable-land-percent-of-land-area-wb-data.html', 'footnotes':'Agricultural land refers to the share of land area that is arable, under permanent crops, and under permanent pastures.'},index=[5564]))
ldown_c.update(pd.DataFrame({'value':100*((28533.2/2)/61974.5), 'general_source':'https://tradingeconomics.com/greenland/arable-land-percent-of-land-area-wb-data.html'},index=[5572]))

larable[['country', 'general_source', 'type', 'value']]


,country,general_source,type,value
1410,djibouti,https://data.un.org/,Land area (thousand hectares),2318.0
1411,djibouti,https://data.un.org/,Arable land (thousand hectares),2.0
1413,djibouti,https://data.un.org/,Arable land (% of total land area),0.1
1650,falkland islands (malvinas),https://data.un.org/,Land area (thousand hectares),1217.0
1973,greenland,https://data.un.org/,Land area (thousand hectares),41045.0
2142,guyana,https://data.un.org/,Land area (thousand hectares),19685.0
2143,guyana,https://data.un.org/,Arable land (thousand hectares),420.0
2146,guyana,https://data.un.org/,Arable land (% of total land area),2.1
2239,iceland,https://data.un.org/,Land area (thousand hectares),10025.0
2240,iceland,https://data.un.org/,Arable land (thousand hectares),121.0


In [13]:
# XI

# III Permanente Crops Faltantes


l_crops = ldown_c[(ldown_c['type'].str.contains('crops|Land area')) & (ldown_c['year']==2016) & (ldown_c['country'].isin(c_falt))]

# [ 'western sahara' 'south sudan']
# 'falkland islands (malvinas)', 'sudan', 'greenland', 'djibouti', 'iceland'    --- LISTO

# Sudan. 
# We have the permanents crops and land area, we make the calculus
addsu = l_crops['value'][(l_crops['country']=='sudan') & (l_crops['type']=='Permanent crops (thousand hectares)')]  
cvvsu = l_crops['value'][(l_crops['country']=='sudan') & (l_crops['type']=='Land area (thousand hectares)')] 
ldown_c.update(pd.DataFrame({'value':100*np.divide(addsu, cvvsu).item(), 'footnotes':'Calculated Value'},index=[5593]))

# Fakland Island
# https://www.cia.gov/library/publications/the-world-factbook/geos/print_fk.html
# agricultural land: 92.4% (2011 est.)
# arable land: 0% (2011 est.) / permanent crops: 0% (2011 est.) / permanent pasture: 92.4% (2011 est.)
# forest: 0% (2011 est.)
# other: 7.6% (2011 est.)
ldown_c.update(pd.DataFrame({'value':0, 'general_source':'https://www.cia.gov/library/publications/the-world-factbook/geos/print_fk.html'},index=[5519]))
ldown_c.update(pd.DataFrame({'value':0, 'general_source':'https://www.cia.gov/library/publications/the-world-factbook/geos/print_fk.html'},index=[5525]))

# Greenland
# https://tradingeconomics.com/greenland/arable-land-percent-of-land-area-wb-data.html
# Agricultural land refers to the share of land area that is arable, under permanent crops, and under permanent pastures.
# Arable land (hectares) 900 ha
# Arable land (hectares per person) 0.016
# Arable land (% of land area) 0.00219 %

ldown_c.update(pd.DataFrame({'value':0.9/2, 'general_source':'https://tradingeconomics.com/greenland/arable-land-percent-of-land-area-wb-data.html', 'footnotes': 'Agricultural land refers to the share of land area that is arable, under permanent crops, and under permanent pastures.'},index=[5537]))
ldown_c.update(pd.DataFrame({'value':0.00219/2, 'general_source':'https://tradingeconomics.com/greenland/arable-land-percent-of-land-area-wb-data.html'},index=[5543]))

# Djibouti
# https://tradingeconomics.com/djibouti/agricultural-land-percent-of-land-area-wb-data.html
# Agricultural land refers to the share of land area that is arable, under permanent crops, and under permanent pastures.
# Arable land (hectares) 2000 ha

ldown_c.update(pd.DataFrame({'value':2/2, 'general_source':'https://tradingeconomics.com/greenland/arable-land-percent-of-land-area-wb-data.html', 'footnotes': 'Agricultural land refers to the share of land area that is arable, under permanent crops, and under permanent pastures.'},index=[5528]))
ldown_c.update(pd.DataFrame({'value':100*(2/2318), 'footnotes':'Calculated Value'},index=[5531]))

# Iceland
# https://tradingeconomics.com/iceland/agricultural-land-percent-of-land-area-wb-data.html
# Agricultural land refers to the share of land area that is arable, under permanent crops, and under permanent pastures.
# Arable land (hectares) 121000 ha 121 th Ha

ldown_c.update(pd.DataFrame({'value':121/2, 'general_source':'https://tradingeconomics.com/iceland/agricultural-land-percent-of-land-area-wb-data.html', 'footnotes': 'Agricultural land refers to the share of land area that is arable, under permanent crops, and under permanent pastures.'},index=[5549]))
ldown_c.update(pd.DataFrame({'value':100*((121/2)/10025),'footnotes':'Calculated Value' },index=[5552]))

# Souht Sudan -
# No clear statistics
ldown_c.update(pd.DataFrame({'value':0, 'footnotes':'No clear statistics founded'},index=[5567]))
ldown_c.update(pd.DataFrame({'value':0,'footnotes':'No clear statistics founded'},index=[5575]))

# Western Sahara
# https://personal.ems.psu.edu/~williams/states/wi.htm
# arable land: 0%
# permanent crops: 0%

ldown_c.update(pd.DataFrame({'value':0, 'general_source':'https://personal.ems.psu.edu/~williams/states/wi.htm'},index=[5599]))
ldown_c.update(pd.DataFrame({'value':0,'general_source':'https://personal.ems.psu.edu/~williams/states/wi.htm' },index=[5602]))

    
l_crops[['country', 'general_source', 'type', 'value']]

,country,general_source,type,value
1410,djibouti,https://data.un.org/,Land area (thousand hectares),2318.0
1650,falkland islands (malvinas),https://data.un.org/,Land area (thousand hectares),1217.0
1973,greenland,https://data.un.org/,Land area (thousand hectares),41045.0
2142,guyana,https://data.un.org/,Land area (thousand hectares),19685.0
2144,guyana,https://data.un.org/,Permanent crops (thousand hectares),50.0
2147,guyana,https://data.un.org/,Permanent crops (% of total land area),0.3
2239,iceland,https://data.un.org/,Land area (thousand hectares),10025.0
2500,jordan,https://data.un.org/,Land area (thousand hectares),8878.0
2502,jordan,https://data.un.org/,Permanent crops (thousand hectares),87.0
2505,jordan,https://data.un.org/,Permanent crops (% of total land area),1.0


In [15]:
# XII 

# Comprobación I: los valores absolutos de las tierras sean menores al land

ldown_cp = ldown_c[ldown_c['year']==2016]

ldown_cp = ldown_cp[['country', 'type', 'value']]

ldown_cp = ldown_cp[~ldown_cp['type'].str.contains('%')]

ldtable = pd.pivot_table(ldown_cp, values='value', index='country', columns='type') 

ldtable = ldtable[['Land area (thousand hectares)', 'Arable land (thousand hectares)', 'Forest cover (thousand hectares)', 'Permanent crops (thousand hectares)' ]]

ldtable['sum'] = ldtable[['Arable land (thousand hectares)','Forest cover (thousand hectares)','Permanent crops (thousand hectares)']].sum(axis=1)


ldtable['ver'] = ldtable['Land area (thousand hectares)']-ldtable['sum']

ldtable['ver_per'] = 100*ldtable['ver']/ldtable['Land area (thousand hectares)'] 
# Porcentaje de la diferencia de tierras que son Arable, Forest and Crops con el Land


ldtable['info']=''
ldtable['info'] = ldtable['info'].mask(ldtable['ver']>=0, 1) 
ldtable['info'] = ldtable['info'].mask(ldtable['ver']<0, 0)
ldtable.sort_values(by='info')


# ---> Si ldtable['info'] = 1, quiere decir que el Land Total es mayor que la suma del resto de los 
# tipos de tierra (Arable, Fores Cover and Permanent Crops); sería Ok. La mayoría son OK.
# No se incluye los 'Important sites for terrestrial biodiversity protected (% of total sites protected)'


# Comprobación II: los valores porcentuales de las tierras sean menores al 100%

ldown_cnp = ldown_c[ldown_c['year']==2016]
ldown_cnp = ldown_cnp[ldown_cnp['type'].str.contains('%')]

ldown_cnp = ldown_cnp[~ldown_cnp['type'].str.contains('Important')]
# quito los 'Important sites for terrestrial biodiversity protected (% of total sites protected)'

# Es posible que los valores de 'Forest cover (% of total land area)' e 'Important sites for 
# terrestrial biodiversity protected (% of total sites protected)' sean conjuntos que se unen, 
# que tienen valores en común.

ldntable = pd.pivot_table(ldown_cnp, values='value', index='country', columns='type') 

ldntable['sum%'] = ldntable.sum(axis=1)

ldntable['info'] = ''
ldntable['info'] = ldntable['info'].mask(ldntable['sum%']>100,1) # Problema
ldntable['info'] = ldntable['info'].mask(ldntable['sum%']<=100,0)


ldntable.sort_values(by='info', ascending=False)
ldntable[ldntable['info']==1].count(), ldntable[ldntable['info']==0].count()
ldntable[ldntable['info']==1].sort_values(ascending=False, by='sum%')

# Quitan los valores de Important Sites; sólo hay 4 regiones cuyos porcentajes son mayores al 100%
# La data is OK!!


type,Arable land (% of total land area),Forest cover (% of total land area),Permanent crops (% of total land area),sum%,info
country,,,,,
micronesia (fed. states of),2.9,91.9,24.3,119.1,1
marshall islands,11.1,70.2,36.1,117.4,1
american samoa,15.0,87.5,9.5,112.0,1
sao tome and principe,9.1,55.8,40.6,105.5,1


In [17]:
# Cambio el orden de las columnas y descargo la data en un .csv

ldown_c = ldown_c[[ 'region', 'country', 'year', 'type', 'value', 'footnotes', 'source', 'general_source']]

ldown_c.head()

ldown_c.to_csv(r'/home/hmdg/Documents/Data_Science/Lands/ldown_c.csv', index = False)

# Hay que:
# Mejorar/automatizar los querys.
# Dividir el código en: I) Inspección de la data, II) cruce de tablas world y landun, III) modificación del cruce para 
# completar la de landun y crear finalmente la de ldown_c
# Revisar si los valores de la mayoría de los casos faltantes vienen de la misma fuente, tratar de automatizar
# o comparar con los existentes....

In [ ]:
df2 = pd.DataFrame(np.array([[1, 2, 'd'], [4, 5, 'e'], [7, 8, 'f']]),
                   columns=['a', 'b', 'c'])
df2

df3 = pd.DataFrame(np.array([[1, 10, 'a'], [4, 12, 'b'], [6, 14, 'c']]),
                   columns=['a', 'b', 'c'])

print('df2\n', df2,'\ndf3\n',df3)

dfn = df2.merge(df3,on='a', how='inner', indicator=True) #'outer', 'right', 'left'
dfn

# Descripcion of Lands Values:
Lands:
Total area excluding area under inland water bodies. The definition of inland water bodies generally includes major
rivers and lakes. Data are expressed in 1000 hectares (Ha).
    
Arable land:
Arable land includes land defined by the FAO as land under temporary crops 
(double-cropped areas are counted once), temporary meadows for mowing or for pasture, 
land under market or kitchen gardens, and land temporarily fallow. Land abandoned as 
a result of shifting cultivation is excluded.

Permanent Crops:
Crops are divided into temporary and permanent crops. Permanent crops are sown or planted once, and then occupy the land for some years and need not be replanted after each annual harvest, such as cocoa, coffee and rubber. This category includes flowering shrubs, fruit trees, nut trees and vines, but excludes trees grown for wood or timber.

Forest:
Area covered with forest, expressed as a percentage of the total land area. 
http://www.fao.org/ag/agn/nutrition/Indicatorsfiles/Agriculture.pdf


# Descripcion of Ecological Footprint (huella ecológica):
El cálculo de la huella ecológica es complejo, y en algunos casos imposible, lo que conforma su principal limitación como indicador; en cualquier caso, existen diversos métodos de estimación a partir del análisis de los recursos que una persona consume y de los residuos que produce. La cantidad de hectáreas utilizadas para urbanizar, generar infraestructuras y centros de trabajo.

- Hectáreas necesarias para proporcionar el alimento vegetal necesario.
- Superficie necesaria para pastos que alimenten al ganado.
- Superficie marina necesaria para producir peces.
- Hectáreas de bosque necesarias para asumir el CO2 que provoca nuestro consumo energético. En este sentido no solo incidiría el grado de eficiencia energética alcanzado sino también las fuentes empleadas para su obtención: a mayor uso de energías renovables, menor huella ecológica.

Desde un punto de vista global, se ha estimado en 1,8 ha2​ la biocapacidad del planeta por cada habitante, o lo que es lo mismo, si tuviéramos que repartir el terreno productivo de la tierra en partes iguales, a cada uno de los más de seis mil millones de habitantes en el planeta, les corresponderían 1,8 hectáreas para satisfacer todas sus necesidades durante un año. Con los datos de 2005, el consumo medio por habitante y año es de 2,7 hectáreas, por lo que, a nivel global, estamos consumiendo más recursos y generando más residuos de los que el planeta puede generar y admitir.

https://es.wikipedia.org/wiki/Huella_ecol%C3%B3gica